<a href="https://colab.research.google.com/github/Kevin-gram/FarmBuddy/blob/main/FarmBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import necessary libraries.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from transformers import AutoTokenizer
from datasets import Dataset

# 📌 Prepare and Tokenize Chatbot Training Data
# This script loads a dataset of user-bot conversations, tokenizes the text using GPT-2,
# and converts it into a Hugging Face Dataset for training a chatbot model.


In [4]:
import json
import random
from transformers import AutoTokenizer
from datasets import Dataset

# Load the dataset
with open("Bigdata.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

# Prepare conversation pairs
conversations = []
for entry in data:
    user_input = entry["prompt"]
    bot_response = entry["completion"]
    conversations.append(f"User: {user_input}\nBot: {bot_response}")

# Load GPT-2 tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Fix: Set padding token explicitly
tokenizer.pad_token = tokenizer.eos_token

# Tokenize conversation data
tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized data into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

# Print a sample to verify everything works
print(dataset[0])

# Now you can train your model with this dataset


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'input_ids': [12982, 25, 1374, 466, 345, 2190, 9875, 14129, 30, 198, 20630, 25, 632, 318, 7151, 284, 3424, 262, 11699, 351, 47375, 290, 788, 11662, 5849, 72, 34, 495, 11662, 319, 262, 5676, 1989, 5403, 257, 1110, 329, 1936, 1528, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

# 📌 Disable Weights & Biases (W&B) Logging  
# Prevents W&B from automatically tracking and logging model training runs.  



In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

# 📌 Load and Fine-Tune GPT-2 Model for Chatbot  
# This script initializes a GPT-2 model, tokenizes conversation data,  
# sets up training parameters, and fine-tunes the model using the Trainer API.  

### 📌 Load Pre-trained GPT-2 Model  
### Initializes a causal language model (GPT-2) for chatbot training.  

### 📌 Tokenize and Prepare Dataset  
### Converts conversation text into tokenized format and creates a Hugging Face dataset.  

### 📌 Define Training Parameters  
### Specifies model training settings, including batch size, epochs, and saving checkpoints.  

### 📌 Set Up Data Collator  
### Helps batch the data correctly by handling padding during training.  

### 📌 Initialize Trainer and Start Training  
### Uses the Trainer API to fine-tune GPT-2 on the prepared dataset.  


In [7]:
import torch
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

training_args = TrainingArguments(
    output_dir="./chatbot_model",
    per_device_train_batch_size=2,
    num_train_epochs=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()


Step,Training Loss
500,2.123500
1000,1.796600
1500,1.649400
2000,1.615700
2500,1.605300
3000,1.420600
3500,1.345600
4000,1.379100
4500,1.387600
5000,1.342800


TrainOutput(global_step=5000, training_loss=1.5666156127929687, metrics={'train_runtime': 1780.6726, 'train_samples_per_second': 5.616, 'train_steps_per_second': 2.808, 'total_flos': 2153617920000000.0, 'train_loss': 1.5666156127929687, 'epoch': 2.0})

In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available

### Save the Fine-Tuned Model and Tokenizer  
### Stores the trained chatbot model and tokenizer for future use.  

### Define Paths for Saving  
### Specifies directories where the model and tokenizer will be stored.  

### Save Model  
### Saves the trained model's weights and configuration.  

### Save Tokenizer  
### Stores the tokenizer, ensuring it matches the trained model.  

### Confirmation Message  
### Prints a success message to confirm that saving was completed.  


In [ ]:
# Save model and tokenizer before using them
model_path = "chatbot_model"
tokenizer_path = "tokenizer"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

print("Model and tokenizer saved successfully!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


### Define Google Drive Save Path  
### Specifies the directory in Google Drive where the model and tokenizer will be saved.  

### Save Model and Tokenizer Locally  
### **Important:** Stores the trained model and tokenizer in the local Colab environment before copying.  

### Copy Files to Google Drive  
### **Important:** Moves the saved model and tokenizer from local storage to Google Drive.  

### Confirmation Message  
### Prints a success message to confirm that saving and copying were completed.  


In [ ]:
import os

# Define the path to save in Google Drive
drive_path = "/content/drive/My Drive/chatbot_model"

# Create paths for model and tokenizer
model_path = os.path.join(drive_path, "model")
tokenizer_path = os.path.join(drive_path, "tokenizer")

# Create directories if they don't exist
os.makedirs(model_path, exist_ok=True)
os.makedirs(tokenizer_path, exist_ok=True)
# Save locally
model.save_pretrained("/content/chatbot_model")
tokenizer.save_pretrained("/content/tokenizer")

# Copy to Google Drive
!cp -r "/content/chatbot_model" "/content/drive/My Drive/"
!cp -r "/content/tokenizer" "/content/drive/My Drive/"

print("Model and tokenizer copied to Google Drive successfully!")


### Define Device for Model  
### **Important:** Checks if a GPU is available and moves the model to the appropriate device (GPU or CPU).  

### Move the Model to the GPU  
### Transfers the model to the selected device (GPU if available, otherwise CPU).  

### Chatbot Response Function  
### **Important:** Tokenizes user input, generates a response, and ensures the model and input are on the same device.  

### Test Chatbot  
### Runs a loop to interact with the chatbot, taking user input and displaying the bot's response.  


In [ ]:
import torch

# Define device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Move the model to the GPU
model = model.to(device)

def chatbot_response(user_input):
    input_text = f"User: {user_input}\nBot:"
    # Tokenize the input and move to the correct device
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    # Generate the response and ensure the input tensor and model are on the same device
    output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)

    # Decode the response and skip special tokens
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

# Test chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    print("Bot:", chatbot_response(user_input))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')